![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [1]:
import pandas as pd

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

# Start coding here...

"""
In order to complete the project you will need to merge the datasets and drop null values, along with answering the questions below.

What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?

    Label products priced up to quartile one as "Budget", quartile two as "Average", quartile three as "Expensive", and quartile four as "Elite".

    Store as a pandas DataFrame called adidas_vs_nike containing the following columns: "brand", "price_label", "num_products", and "mean_revenue". All numeric values should be rounded to two decimal places.

Do any differences exist between the word count of a product's description and its mean rating?

    Split product description length into bins of 100 characters and calculate the average rating and number of reviews.
    Store the results as a pandas DataFrame called description_lengths containing the following columns: "description_length", "mean_rating", "num_reviews", again rounding numeric values to two decimal places.

How does the volume of products and median revenue vary between clothing and footwear?

    Search "description" for "shoe*", "trainer*", or "foot*" and use the results to calculate the number of footwear products versus clothing products sold by the company and the median revenue for each of the two product types.
    Create a pandas DataFrame called product_types containing the following columns: "num_clothing_products", "median_clothing_revenue", "num_footwear_products", "median_footwear_revenue".
"""

#merge data
merged_df = info.merge(finance, on="product_id")
merged_df = merged_df.merge(reviews, on="product_id")
merged_df = merged_df.merge(brands, on="product_id")
#drop nulls
merged_df.dropna(inplace=True)

###What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?

#price labels based on listing_price quartiles
#Label products priced up to quartile one as "Budget", quartile two as "Average", quartile three as "Expensive", and quartile four as "Elite".

merged_df["price_label"] = pd.qcut(merged_df["listing_price"], q=4, labels=["Budget", "Average", "Expensive", "Elite"])

#group by brand and price_label to get volume and mean revenue
#Store as a pandas DataFrame called adidas_vs_nike containing the following columns: "brand", "price_label", "num_products", and "mean_revenue". All numeric values should be rounded to two decimal places.

adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(
    num_products=("price_label", "count"), 
    mean_revenue=("revenue", "mean")
).round(2)
print(adidas_vs_nike)

###Do any differences exist between the word count of a product's description and its mean rating?

#Split product description length into bins of 100 characters and calculate the average rating and number of reviews.
#the length of each description
merged_df["description_length"] = merged_df["description"].str.len()
#upper description length limits
lengthes = [0, 100, 200, 300, 400, 500, 600, 700]
#description length labels
labels = ["100", "200", "300", "400", "500", "600", "700"]
#into bins
merged_df["description_length"] = pd.cut(merged_df["description_length"], bins=lengthes, labels=labels)
#Store the results as a pandas DataFrame called description_lengths containing the following columns: "description_length", "mean_rating", "num_reviews", again rounding numeric values to two decimal places.
#group by the bins
description_lengths = merged_df.groupby("description_length", as_index=False).agg(
    mean_rating=("rating", "mean"), 
    num_reviews=("reviews", "count")
).round(2)
print(description_lengths)

###How does the volume of products and median revenue vary between clothing and footwear?

#Search "description" for "shoe*", "trainer*", or "foot*" and use the results to calculate the number of footwear products versus clothing products sold by the company and the median revenue for each of the two product types.
#list of footwear keywords
mylist = "shoe*|trainer*|foot*"
#filter for footwear products
shoes = merged_df[merged_df["description"].str.contains(mylist)]
#filter for clothing products
clothing = merged_df[~merged_df.isin(shoes["product_id"])]
#remove null product_id values from clothing
clothing.dropna(inplace=True)
#create product_types DataFrame
product_types = pd.DataFrame({"num_clothing_products": len(clothing), 
                              "median_clothing_revenue": clothing["revenue"].median(), 
                              "num_footwear_products": len(shoes), 
                              "median_footwear_revenue": shoes["revenue"].median()}, 
                              index=[0])
print(product_types)

    brand price_label  num_products  mean_revenue
0  Adidas      Budget           574       2015.68
1  Adidas     Average           655       3035.30
2  Adidas   Expensive           759       4621.56
3  Adidas       Elite           587       8302.78
4    Nike      Budget           357       1596.33
5    Nike     Average             8        675.59
6    Nike   Expensive            47        500.56
7    Nike       Elite           130       1367.45
  description_length  mean_rating  num_reviews
0                100         2.26            7
1                200         3.19          526
2                300         3.28         1785
3                400         3.29          651
4                500         3.35          118
5                600         3.12           15
6                700         3.65           15
   num_clothing_products  ...  median_footwear_revenue
0                    478  ...                   3073.3

[1 rows x 4 columns]
